In [67]:
#Código do Joao Pedro
# import os
# import cv2
 
# def aplica_filtros(pasta_in, pasta_out):
#     for pasta_nome in os.listdir(pasta_in):
#         caminho_pasta = os.path.join(pasta_in, pasta_nome)
       
#         pasta_saida = os.path.join(pasta_out, pasta_nome)
#         os.makedirs(pasta_saida, exist_ok=True)
       
#         for nome_arquivo in os.listdir(caminho_pasta):
#             file_path = os.path.join(caminho_pasta, nome_arquivo)
           
#             if nome_arquivo.lower().endswith('.jpg') or nome_arquivo.lower().endswith('.jpeg'):
 
#                 img = cv2.imread(file_path, 0)
               
#                 img_gaus = cv2.GaussianBlur(img, (3, 3), 1)
               
#                 _, img_otsu = cv2.threshold(img_gaus, 175, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
               
#                 cv2.imwrite(os.path.join(pasta_saida, f"{nome_arquivo.split('.')[0]}.jpg"), img_otsu)
 
# pasta_in = "dados"
# pasta_out = "dados_modificados"
 
# aplica_filtros(pasta_in, pasta_out)
#
import os
import cv2
import matplotlib.pyplot as plt


pasta_origem = "C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B"

pasta_destino = "C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/resultado_pós_suavizacao_e_otsu"

arq = os.listdir(pasta_origem)

for i in arq:
    #verificando se o que está sendo lido é uma imagem desejada
    if i.lower().startswith(('cgtoll10b')):
        caminho_origem = os.path.join(pasta_origem, i)
        img = cv2.imread(caminho_origem, cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"Erro ao ler a imagem: {caminho_origem}")
        else:
            print(f"Imagem lida com sucesso: {caminho_origem}")

            gaussiana = cv2.GaussianBlur(img, (3, 3), 1)
            _, otsu = cv2.threshold(gaussiana, 175, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            cv2.imwrite(os.path.join(pasta_destino, f"{i.split('.')[0]}.jpg"), otsu)

print(caminho_origem)

Erro ao ler a imagem: C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch00.tif
Erro ao ler a imagem: C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch01.tif
Erro ao ler a imagem: C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch02.tif
Erro ao ler a imagem: C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series011_z0_ch00.tif
Erro ao ler a imagem: C:/Users/lazar/Universidade Federal de Uberlândia/Bruno Augusto Nassif Travencolo - Lazaro/data/set20210322/CgToll10B\CgToll10B_Ra

In [7]:

import os
import cv2
import matplotlib.pyplot as plt

pasta_origem = 'C:\\Users\\lazar\\Universidade Federal de Uberlândia\\Bruno Augusto Nassif Travencolo - Lazaro\\data\\set20210322\\CgToll10B'
pasta_destino = 'C:\\Users\\lazar\\Universidade Federal de Uberlândia\\Bruno Augusto Nassif Travencolo - Lazaro\\data\\set20210322\\resultado_pós_suavizacao_e_otsu'

arquivos = os.listdir(pasta_origem)

for i in arquivos:
    # Verificando se o que está sendo lido é uma imagem desejada
    if i.lower().endswith(('.tiff')):
        caminho_origem = os.path.join(pasta_origem, i)

        img = cv2.imread(caminho_origem)

        if img is None:
            print(f"Erro ao ler a imagem: {caminho_origem}")
        else:
            print(f"Imagem lida com sucesso: {caminho_origem}")

            # Aplicar filtros aqui (gaussian blur, thresholding, etc.)
            img_gaus = cv2.GaussianBlur(img, (3, 3), 1)
            _, img_otsu = cv2.threshold(img_gaus, 175, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Exibir a imagem original (opcional)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Converter de BGR para RGB
            plt.title(f"Original: {i}")
            plt.show()

            # Exibir a imagem processada (opcional)
            plt.imshow(img_otsu, cmap='gray')
            plt.title(f"Processada")
            plt.show()

            # Salvar a imagem processada na pasta de destino
            nome_arquivo_saida = f"{i.split('.')[0]}_processada.jpg"
            caminho_destino = os.path.join(pasta_destino, nome_arquivo_saida)
            cv2.imwrite(caminho_destino, img_otsu)

            print(f"Imagem processada salva em: {caminho_destino}")